---

(🎧) 표시가 되어 있는 부분은 Wire 페이지에서 오디오 코드 해설을 제공하고 있습니다.

---

# 실습 1. 단어사전 만들기

단어사전을 만드는 것은 NLU 모델링의 시작이자 가장 중요한 부분 중 하나입니다.   
모델은 단어사전에 있는 토큰만을 인식할 수 있고, 이 사전을 토대로 Language와 태스크를 배우기 때문입니다.


<b>학습 목표:    
- 자연어처리의 기본인 토크나이징의 개념을 이해한다.
- 딥러닝을 위한 단어사전을 구축할 때 유의할 점을 파악한다.</b>







## #0. 실습 준비하기
구글 드라이브에 NLP 폴더를 모두 생성해두셨나요?   
단어사전을 만든 후 드라이브의 NLP 폴더에 저장하기 위해 Colab을 구글 드라이브와 연동하겠습니다.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### > 라이브러리 설치

konlpy라는 한국어 자연어처리 라이브러리를 사용해 자연어를 형태소 단위로 분석할 수 있습니다.

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 45.6MB/s 
     |████████████████████████████████| 92kB 3.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


#### > 데이터 준비하기
실습에서 사용할 데이터셋을 다운받아 읽어오는 부분입니다.   

Source Data :
- 한국어 위키백과 단락 일부
- KorQuAD 1.0 데이터 본문 활용

In [ ]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json

import json

with open("/content/KorQuAD_v1.0_train.json") as f:
  data = json.loads(f.read())

PARAS = []
for dat in data["data"]:
  for para in dat["paragraphs"]:
    PARAS.append(para["context"])

--2020-11-16 06:31:10--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  72.5MB/s    in 0.5s    

2020-11-16 06:31:11 (72.5 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]



👉 다운로드한 데이터셋의 첫 번째 문단을 프린트해보겠습니다.    괴테의 파우스트와 관련된 위키백과 페이지네요.   

👉 위키백과는 이러한 문학 뿐만 아니라 정치, 경제, 사회, 과학 전반을 아우르는 다양한 주제가 포함되어 있기 때문에   
토큰 임베딩 사전학습에 사용할 수 있는 좋은 코퍼스 중 하나라고 볼 수 있습니다.   

👉 총 9,681개의 문단이 데이터셋에 포함되어 있는 것도 확인할 수 있습니다.

In [ ]:
print("** Number of Paragraphs:", len(PARAS))
PARAS[0]

** Number of Paragraphs: 9681


'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

## #1. 토크나이징



<img src = "https://github.com/seungyounglim/temporary/blob/master/fig_step1.PNG?raw=true">

- konply : 한국어 자연어처리 관련 패키지
- konply의 Komoran tagger을 이용해 형태소 분석 실행
   
   
이제 본격적으로 konlpy 패키지를 이용해 형태소분석을 실시해보겠습니다.   
konlpy에는 Komoran, Hannanum, Kkma, Okt 등 다양한 분석기가 포함되어 있습니다.   
형태소 분석기를 호출한 후 tokenizer.morphs() 함수를 통해 분석 결과를 얻을 수 있습니다.

In [ ]:
from konlpy.tag import Komoran, Hannanum, Kkma, Okt
komoran = Komoran()
hannanum = Hannanum()
kkma = Kkma()
okt = Okt()

# 문장, 형태소분석기를 인풋으로 받아 쪼개진 문장을 리턴하는 함수 정의
def tokenize(sentence, tokenizer):
  return tokenizer.morphs(sentence)

In [ ]:
sentence = "형태소 분석기마다 쪼개진 문장 특성이 조금씩 달라요"
print("한나눔:", tokenize(sentence, hannanum))
print("꼬꼬마:", tokenize(sentence, kkma))
print("코모란:", tokenize(sentence, komoran))
print("트위터:", tokenize(sentence, okt))

한나눔: ['형태소', '분석기', '마다', '쪼개', '어', '지', 'ㄴ', '문장', '특성', '이', '조금씩', '달라요']
꼬꼬마: ['형태소', '분석기', '마다', '쪼개지', 'ㄴ', '문장', '특성', '이', '조금씩', '닿', 'ㄹ라요']
코모란: ['형태소', '분석기', '마다', '쪼개', '어', '지', 'ㄴ', '문장', '특성', '이', '조금', '씩', '다르', '아요']
트위터: ['형태소', '분석', '기마', '다', '쪼개진', '문장', '특성', '이', '조금씩', '달라', '요']


형태소분석기마다 분석 결과가 조금씩 다른 것을 볼 수 있습니다.   

👉한나눔, 꼬꼬마, 코모란 분석기는 원형복원을 하기 때문에, 형태소의 원형이 복원된 결과를 리턴합니다.   
👉트위터분석기는 들어온 문장을 형태소 단위로 쪼개서 그대로 리턴하고 있습니다.   

우리는 실습에서 속도가 비교적 빠른 komoran 분석기를 사용할 것입니다.   
코모란 분석기를 사용해서 문장을 인풋으로 받아 쪼개진 리스트를 리턴하는 tokenize 함수를 직접 작성해보세요.

<font color = "red">[MISSION] 코모란 분석기를 이용해 문장을 인풋으로 받아 형태소 분석된 리스트를 반환하는 tokenize 함수를 구현해보세요</font>

In [ ]:
komoran = Komoran()

def tokenize(sentence):
  return komoran.morphs(sentence)

tokenize("미션 완료!!")

['미션', '완료', '!!']

## #2. 단어 사전 만들기

👏자연어가 들어올 때 형태소 분석을 완료하고, 분석된 결과를 리턴할 수 있는 함수를 정의했습니다.   

이제 분석된 형태소를 모델이 처리할 수 있는 정수 인덱스로 변환해야 합니다.   
그리고 이 과정에는 단어사전이 반드시 필요하지요.

🙆‍♀️단어사전을 만들 때 고려해야 할 사항, 기억하시나요?

1.  몇 개의 토큰을 사전에 포함할 것인가?
2.  배치 연산을 위해 필요한 Padding([PAD])과 Out of vocabulary([UNK]) 토큰을 반드시 포함해야 함

이 두 가지를 염두에 두고, 이제 단어사전을 만들어보겠습니다.


#### Step 1. 학습 코퍼스에 있는 형태소 토큰의 Frequency 체크하기 

먼저 학습 코퍼스에 있는 토큰이 어떤 것들이 있는지, 몇 번씩 등장하는지 체크해야 합니다.   

모든 토큰을 단어사전에 포함하면 자칫 임베딩 크기가 너무 커질 수 있기 때문에    
빈도수가 낮은 토큰은 제외하고 싶을 수 있기 때문이지요.   


우리는 총 9,681개 위키백과 문단들을 학습 코퍼스로 학습했는데요,   
이 중에서 8,000개 문단은 학습 데이터로, 나머지는 검증용 데이터로 나눠서 모델링에 사용하겠습니다.

In [ ]:
# 전체 문단을 Train과 Validation으로 나누기
import random
random.seed(1)
random.shuffle(PARAS)

PARAS_tr = PARAS[:8000]
PARAS_dev = PARAS[8000:]

print("Train: {} | Val: {}".format(len(PARAS_tr), len(PARAS_dev)))

Train: 8000 | Val: 1681


In [ ]:
## 분할한 데이터셋 구글 드라이브에 저장
import json
with open("/content/gdrive/My Drive/NLP/CBOW_train_paras.json" , 'w') as f:
  f.write(json.dumps(PARAS_tr))
with open("/content/gdrive/My Drive/NLP/CBOW_dev_paras.json" , 'w') as f:
  f.write(json.dumps(PARAS_dev))

다음으로 학습 코퍼스의 각각의 문단을 형태소 단위로 쪼개고, 등장한 형태소 토큰의 빈도수를 체크하겠습니다.   
collections 라이브러리의 Counter 기능을 사용하면 간단하게 할 수 있습니다. (🎧) 

In [ ]:
# 훈련 셋에 있는 Paragraph를 형태소로 쪼개고,토큰의 빈도를 체크하기 
from tqdm import tqdm
from collections import Counter

vocab_freq = Counter()
TOKEN_PARAS_tr = []
for i,para in tqdm(enumerate(PARAS_tr)):
  tokenized_para = tokenize(para)
  TOKEN_PARAS_tr.append(tokenized_para)
  if i == 0: print("\n... tokenized_para 예시:", tokenized_para)
  for word in tokenized_para:
    vocab_freq[word] += 1

4it [00:00, 34.51it/s]


... tokenized_para 예시: ['차량', '정비', '소가', '있', '는', '패턴', '에', '만', '있', '다', '.', '화가', '로', '추정', '되', '는', '중립', '민간인', '노숙자', 'NPC', '가', '1', '명', '있', '고', '음식', '을', '구걸', '하', '는데', ',', '종류', '에', '상관', '없이', '음식', '을', '주', '면', '지하', '에', '숨기', '어', '지', 'ㄴ', '물품', '더미', '의', '존재', '를', '알리', '어', '주', '고', '사기', '가', '올라가', 'ㄴ다', '.', '노숙자', '가', '알리', '어', '주', 'ㄴ', '물품', '더미', '에서', '보석', '과', '알콜', '을', '각각', '1', '개', '씩', '얻', '을', '수', '있', '다', '.', '돕', '지', '않', '으면', '몇', '차례', '방문', '뒤', '결국', '노숙자', '는', '굶', '어', '죽', '어', '그', '시체', '를', '가져오', 'ㄹ', '수', '있', '다', '.', '노숙자', '가', '나타나', 'ㄴ', '뒤', '누', '군가', '를', '찾', '아', '헤매', '는', '사람', '이', '오', '는', '이벤트', '가', '확률', '적', '으로', '발생', '하', 'ㄴ다', '.', '노숙자', '는', '죽이', '었', '을', '때', '얻', '을', '수', '있', '는', '아이템', '이', '가치', '가', '높', '지', '않', '기', '때문', '에', '되', '도록', '돕', '는', '것', '이', '낫', '다', '.', '간혹', '노숙자', '에게', '음식', '을', '주', '었', '을', '때', ',', '노숙자', '가', '감사', '인사', '와', '함께', '따라오', '라고'

8000it [01:09, 115.72it/s]


In [ ]:
print("Total tokens :", len(vocab_freq))

Total tokens : 71102


#### Step 2. 단어의 Frequency에 따라 단어사전에 포함할 토큰 골라내기
👉학습 코퍼스에는 총 70000개 이상의 토큰이 포함되어 있네요.

코퍼스 자체가 크지 않다보니 이 토큰을 모두 포함해 단어사전을 만들어도 무리는 없는 숫자입니다.   
하지만 연습을 위해, 이 중 70000개 토큰만을 추려서 사전을 만들겠습니다.   
Counter의 most_common 기능을 이용해서 가장 많이 등장한 토큰과 가장 적게 등장한 토큰을 확인해보겠습니다.

In [ ]:
N = 70000
most_common = vocab_freq.most_common(len(vocab_freq))

print("가장 많이 나온 10개 토큰:", most_common[:10])
print("가장 적게 나온 10개 토큰:",most_common[-10:])

가장 많이 나온 10개 토큰: [('하', 85964), ('이', 62446), ('.', 58144), ('의', 51015), ('는', 50509), ('을', 49238), ('다', 48650), ('ㄴ', 45041), ('에', 44015), (',', 42909)]
가장 적게 나온 10개 토큰: [('카리브 판', 1), ('아루바', 1), ('연방 속지', 1), ('수리남', 1), ('에콰도르령', 1), ('갈라파고스 제도', 1), ('칠레령', 1), ('로빈슨 크루소', 1), ('칠로에 섬', 1), ('티에라델푸에고 제도', 1)]


👉조사나 마침표 같은 토큰이 가장 많이 등장했고, 한 두번씩만 등장한 고유명사들이 가장 적게 나온 토큰으로 꼽혔습니다.


이제 가장 자주 등장한 N=70000개 토큰만을 포함해 vocabulary_set을 만들겠습니다.

In [ ]:
vocabulary_set = [s[0] for s in most_common[:N]]
print(vocabulary_set[:10])

['하', '이', '.', '의', '는', '을', '다', 'ㄴ', '에', ',']


최종 단어사전은 위에서 골라낸 토큰과 더불어 [PAD]와 [UNK] 토큰을 포함해야겠지요?

딥러닝에서 [PAD]토큰은 0번 인덱스를 사용하는 것이 일반적입니다.   

따라서 0번째 자리에 [PAD], 1번째에 [UNK],    
2번째부터는 위에서 골라낸 vocabulary_set에 있는 단어들이 차례로 오는 vocabulary_list를 만들겠습니다.   

In [ ]:
## 단어 사전 생성을 위해 [PAD] , [UNK] 토큰을 포함하는 vocabulary_list 생성
vocabulary_list = ["[PAD]", "[UNK]"]
vocabulary_list.extend(vocabulary_set)

print("최종 단어 개수: {}개".format(len(vocabulary_list)) )

최종 단어 개수: 70002개


In [ ]:
vocabulary_list[:10]

['[PAD]', '[UNK]', '하', '이', '.', '의', '는', '을', '다', 'ㄴ']

#### Step 3. 만들어진 단어사전 저장하기

👉[PAD]와 [UNK] 토큰을 포함해 총 70002개의 토큰을 포함한 단어사전이 완성되었습니다.   
이제 이 사전을 구글드라이브에 저장하여 다음 모듈에서 활용하도록 하겠습니다.   

In [ ]:
import io
out_m = io.open("/content/gdrive/My Drive/NLP/meta.tsv" , 'w', encoding='utf-8')
for num, word in enumerate(vocabulary_list):
  out_m.write(word + "\n")
out_m.close()